In [2]:
"""
Pipe line to process simulation data and extract features for the inverse modelling task, and save them for future use.
The purpose is to cut down on the time it takes to process the data and extract features.
"""

from pathlib import Path
import json
import pandas as pd
from inverse_modelling_tfo.data_pipelines.fetal_conc_groups import dan_iccps_pencil1, generate_grouping_from_config
from inverse_modelling_tfo.data import config_based_normalization
from inverse_modelling_tfo.data.intensity_interpolation import (
    interpolate_exp,
    get_interpolate_fit_params,
    exp_piecewise_affine,
)
from inverse_modelling_tfo.data.interpolation_function_zoo import *
from inverse_modelling_tfo.features.build_features import (
    FetalACFeatureBuilder,
    RowCombinationFeatureBuilder,
    TwoColumnOperationFeatureBuilder,
    FetalACbyDCFeatureBuilder,
    LogTransformFeatureBuilder,
    ConcatenateFeatureBuilder,
)
from inverse_modelling_tfo.features.data_transformations import (
    LongToWideIntensityTransformation,
    ToFittingParameterTransformation,
)

# Data Setup
# ==========================================================================================
out_dest = Path(r'/home/rraiyan/personal_projects/tfo_inverse_modelling/data/processed_data') / 'sythentic_ppg_base.pkl'
# out_dest = Path(__file__).parent.parent.parent / "data" / "processed_data" / "pulastion_ratio.pkl"
# out_dest = Path(__file__).parent.parent.parent / "data" / "processed_data" / "processed1_max_long_range.pkl"
config_dest = out_dest.with_suffix(".json")

in_src = Path(r'/home/rraiyan/simulations/tfo_sim/data/compiled_intensity/synthetic_ppg_base.pkl')
# in_src = Path(r"/home/rraiyan/simulations/tfo_sim/data/compiled_intensity/weitai_data.pkl")
config_src = in_src.with_suffix(".json")

data = pd.read_pickle(in_src)
config_based_normalization(data, config_src)

In [3]:
rounding_points = 2     # Since we are comparing floats, better to round them up to avoid comparison errors
with config_src.open("r") as file:
    config = json.load(file)

fetal_grouping_map = config["fconc_centers"]  # The key will be strings, needs to be converted to float
maternal_grouping_map = config["mconc_centers"]
fetal_grouping_map = {round(float(k), rounding_points): v for k, v in fetal_grouping_map.items()}
maternal_grouping_map = {round(float(k), rounding_points): v for k, v in maternal_grouping_map.items()}

In [4]:
maternal_grouping_map

{14.0: 0, 13.65: 0}

$$
\mu_a = c \times \epsilon (S)
$$

$$
I = \int exp(-\mu_a * L) p(L) dL
$$

In [5]:
# Data Processing
# ==========================================================================================
data = data.drop(columns="Uterus Thickness")

# Interpolate intensity to remove noise
# data = interpolate_exp(data, weights=(1, 0.6), interpolation_function=exp_piecewise_affine, break_indices=[4, 12, 20])
# data["Intensity"] = data["Interpolated Intensity"]  # Replace OG intensity with interpolated intensity
# data = data.drop(columns="Interpolated Intensity")  # Cleanup

# Define data transformers
data_transformer = LongToWideIntensityTransformation()
# fitting_param_transformer = ToFittingParameterTransformation()

# Transform data
# fitting_params = fitting_param_transformer.transform(data)
data = data_transformer.transform(data)
labels = data_transformer.get_label_names()
intensity_columns = data_transformer.get_feature_names()

# Cleanup
data.dropna(inplace=True)

# Create fetal conc. grouping column - used for generating the AC component/which rows to choose for pairing
data["FconcCenters"] = data["Fetal Hb Concentration"].round(rounding_points).map(fetal_grouping_map)
data["MconcCenters"] = data["Maternal Hb Concentration"].round(rounding_points).map(maternal_grouping_map)
data.head()

,Maternal Wall Thickness,Maternal Hb Concentration,Maternal Saturation,Fetal Hb Concentration,Fetal Saturation,10_2.0,15_2.0,19_2.0,24_2.0,28_2.0,...,64_1.0,68_1.0,72_1.0,77_1.0,81_1.0,86_1.0,90_1.0,94_1.0,FconcCenters,MconcCenters
0,4.0,13.65,0.92,14.625,0.20,0.000008,3.777409e-07,3.802294e-08,3.301643e-09,5.186053e-10,...,7.097240e-11,4.129214e-11,2.446615e-11,1.472874e-11,7.711925e-12,5.138559e-12,3.321740e-12,2.081280e-12,0,0
1,4.0,13.65,0.92,14.625,0.22,0.000008,3.777107e-07,3.801161e-08,3.298559e-09,5.176437e-10,...,7.166575e-11,4.170675e-11,2.471426e-11,1.486830e-11,7.793113e-12,5.194178e-12,3.355441e-12,2.099397e-12,0,0
2,4.0,13.65,0.92,14.625,0.24,0.000008,3.776809e-07,3.800057e-08,3.295461e-09,5.166943e-10,...,7.237513e-11,4.213112e-11,2.496787e-11,1.501101e-11,7.876194e-12,5.251069e-12,3.389916e-12,2.117923e-12,0,0
3,4.0,13.65,0.92,14.625,0.26,0.000008,3.776512e-07,3.798942e-08,3.292384e-09,5.157478e-10,...,7.310091e-11,4.256521e-11,2.522728e-11,1.515687e-11,7.961257e-12,5.309272e-12,3.425200e-12,2.136865e-12,0,0
4,4.0,13.65,0.92,14.625,0.28,0.000008,3.776234e-07,3.797845e-08,3.289319e-09,5.148151e-10,...,7.384454e-11,4.300953e-11,2.549286e-11,1.530611e-11,8.048381e-12,5.368835e-12,3.461325e-12,2.156236e-12,0,0


In [6]:
labels = labels + ["FconcCenters", "MconcCenters"] # This new column grouping should also be treated as a label
# fitting_params['FconcCenters'] = data['FconcCenters']
fixed_columns = [
    "Maternal Wall Thickness",
    "Maternal Saturation",
    "Fetal Saturation",
    "FconcCenters",
    "MconcCenters"
]
# These groups should be combined into a single row
data_test = data.groupby(fixed_columns)
data_test.groups

{(4.0, 0.92, 0.2, 0, 0): [0, 26, 468, 494], (4.0, 0.92, 0.22, 0, 0): [1, 27, 469, 495], (4.0, 0.92, 0.24, 0, 0): [2, 28, 470, 496], (4.0, 0.92, 0.26, 0, 0): [3, 29, 471, 497], (4.0, 0.92, 0.28, 0, 0): [4, 30, 472, 498], (4.0, 0.92, 0.3, 0, 0): [5, 31, 473, 499], (4.0, 0.92, 0.32, 0, 0): [6, 32, 474, 500], (4.0, 0.92, 0.33999999999999997, 0, 0): [7, 33, 475, 501], (4.0, 0.92, 0.36, 0, 0): [8, 34, 476, 502], (4.0, 0.92, 0.38, 0, 0): [9, 35, 477, 503], (4.0, 0.92, 0.39999999999999997, 0, 0): [10, 36, 478, 504], (4.0, 0.92, 0.42, 0, 0): [11, 37, 479, 505], (4.0, 0.92, 0.43999999999999995, 0, 0): [12, 38, 480, 506], (4.0, 0.92, 0.45999999999999996, 0, 0): [13, 39, 481, 507], (4.0, 0.92, 0.48, 0, 0): [14, 40, 482, 508], (4.0, 0.92, 0.49999999999999994, 0, 0): [15, 41, 483, 509], (4.0, 0.92, 0.52, 0, 0): [16, 42, 484, 510], (4.0, 0.92, 0.54, 0, 0): [17, 43, 485, 511], (4.0, 0.92, 0.5599999999999999, 0, 0): [18, 44, 486, 512], (4.0, 0.92, 0.58, 0, 0): [19, 45, 487, 513], (4.0, 0.92, 0.59999999

In [7]:
data.iloc[[220, 231, 330, 341], :]

,Maternal Wall Thickness,Maternal Hb Concentration,Maternal Saturation,Fetal Hb Concentration,Fetal Saturation,10_2.0,15_2.0,19_2.0,24_2.0,28_2.0,...,64_1.0,68_1.0,72_1.0,77_1.0,81_1.0,86_1.0,90_1.0,94_1.0,FconcCenters,MconcCenters
220,4.0,13.65,0.96,14.625,0.44,0.000008,3.716406e-07,3.723188e-08,3.206999e-09,4.992161e-10,...,8.598278e-11,5.012340e-11,2.972817e-11,1.773219e-11,9.441666e-12,6.314028e-12,4.044700e-12,2.483186e-12,0,0
231,4.0,13.65,0.96,14.625,0.66,0.000008,3.713428e-07,3.712104e-08,3.176733e-09,4.899702e-10,...,9.872292e-11,5.769893e-11,3.421924e-11,2.024853e-11,1.093661e-11,7.322800e-12,4.659973e-12,2.809571e-12,0,0
330,4.0,13.65,0.98,14.625,0.56,0.000008,3.686310e-07,3.684736e-08,3.161499e-09,4.900188e-10,...,9.554713e-11,5.574751e-11,3.306770e-11,1.963131e-11,1.054764e-11,7.061526e-12,4.503881e-12,2.736157e-12,0,0
341,4.0,13.65,0.98,15.000,0.26,0.000008,3.688947e-07,3.694572e-08,3.188514e-09,4.983002e-10,...,7.934858e-11,4.609556e-11,2.732478e-11,1.640554e-11,8.648713e-12,5.771122e-12,3.720780e-12,2.317346e-12,0,0


In [8]:
# Do the combination 
fb1 = RowCombinationFeatureBuilder(intensity_columns, fixed_columns, ["Fetal Hb Concentration", "Maternal Hb Concentration"], "comb", 4)
data = fb1(data)
print(len(data))

7254


In [9]:
# Recheck the grouping - now we should have a single row per group
data_test = data.groupby(fixed_columns)
data_test.groups

{(4.0, 0.92, 0.2, 0.0, 0.0): [0], (4.0, 0.92, 0.22, 0.0, 0.0): [1], (4.0, 0.92, 0.24, 0.0, 0.0): [2], (4.0, 0.92, 0.26, 0.0, 0.0): [3], (4.0, 0.92, 0.28, 0.0, 0.0): [4], (4.0, 0.92, 0.3, 0.0, 0.0): [5], (4.0, 0.92, 0.32, 0.0, 0.0): [6], (4.0, 0.92, 0.33999999999999997, 0.0, 0.0): [7], (4.0, 0.92, 0.36, 0.0, 0.0): [8], (4.0, 0.92, 0.38, 0.0, 0.0): [9], (4.0, 0.92, 0.39999999999999997, 0.0, 0.0): [10], (4.0, 0.92, 0.42, 0.0, 0.0): [11], (4.0, 0.92, 0.43999999999999995, 0.0, 0.0): [12], (4.0, 0.92, 0.45999999999999996, 0.0, 0.0): [13], (4.0, 0.92, 0.48, 0.0, 0.0): [14], (4.0, 0.92, 0.49999999999999994, 0.0, 0.0): [15], (4.0, 0.92, 0.52, 0.0, 0.0): [16], (4.0, 0.92, 0.54, 0.0, 0.0): [17], (4.0, 0.92, 0.5599999999999999, 0.0, 0.0): [18], (4.0, 0.92, 0.58, 0.0, 0.0): [19], (4.0, 0.92, 0.5999999999999999, 0.0, 0.0): [20], (4.0, 0.92, 0.6199999999999999, 0.0, 0.0): [21], (4.0, 0.92, 0.6399999999999999, 0.0, 0.0): [22], (4.0, 0.92, 0.6599999999999999, 0.0, 0.0): [23], (4.0, 0.92, 0.679999999999

In [10]:
group_lengths = [len(data_test.groups[group_key]) for group_key in data_test.groups.keys()]
assert all([x == 1 for x in group_lengths]), "Grouping is not correct"

In [11]:
fb1.get_label_names()

['Maternal Wall Thickness',
 'Maternal Saturation',
 'Fetal Saturation',
 'FconcCenters',
 'MconcCenters',
 'Fetal Hb Concentration 1',
 'Maternal Hb Concentration 1',
 'Fetal Hb Concentration 2',
 'Maternal Hb Concentration 2',
 'Fetal Hb Concentration 3',
 'Maternal Hb Concentration 3',
 'Fetal Hb Concentration 4',
 'Maternal Hb Concentration 4']

In [12]:
data.head()

,10_2.0_1,15_2.0_1,19_2.0_1,24_2.0_1,28_2.0_1,33_2.0_1,37_2.0_1,41_2.0_1,46_2.0_1,50_2.0_1,...,FconcCenters,MconcCenters,Fetal Hb Concentration 1,Maternal Hb Concentration 1,Fetal Hb Concentration 2,Maternal Hb Concentration 2,Fetal Hb Concentration 3,Maternal Hb Concentration 3,Fetal Hb Concentration 4,Maternal Hb Concentration 4
0,0.000008,3.777409e-07,3.802294e-08,3.301643e-09,5.186053e-10,1.109698e-10,3.766455e-11,1.537745e-11,6.925765e-12,3.158632e-12,...,0.0,0.0,14.625,13.65,15.0,13.65,14.625,14.0,15.0,14.0
1,0.000008,3.777107e-07,3.801161e-08,3.298559e-09,5.176437e-10,1.106862e-10,3.756772e-11,1.534023e-11,6.910129e-12,3.151476e-12,...,0.0,0.0,14.625,13.65,15.0,13.65,14.625,14.0,15.0,14.0
2,0.000008,3.776809e-07,3.800057e-08,3.295461e-09,5.166943e-10,1.104055e-10,3.747201e-11,1.530332e-11,6.894601e-12,3.144354e-12,...,0.0,0.0,14.625,13.65,15.0,13.65,14.625,14.0,15.0,14.0
3,0.000008,3.776512e-07,3.798942e-08,3.292384e-09,5.157478e-10,1.101264e-10,3.737716e-11,1.526655e-11,6.879207e-12,3.137289e-12,...,0.0,0.0,14.625,13.65,15.0,13.65,14.625,14.0,15.0,14.0
4,0.000008,3.776234e-07,3.797845e-08,3.289319e-09,5.148151e-10,1.098495e-10,3.728224e-11,1.523030e-11,6.863911e-12,3.130267e-12,...,0.0,0.0,14.625,13.65,15.0,13.65,14.625,14.0,15.0,14.0


In [13]:
# Create Config file
# ==========================================================================================
# NOT AUTOGENRATED! MUST BE DONE MANUALLY FOR EACH PIPELINE
config = {
    "labels": fb1.get_label_names(),
    "features": fb1.get_feature_names(),
    "feature_builder_txt": str(fb1),
    "preprocessing_description": "Detector Normalization -> Long to Wide -> Row Combination -> Keep 4 intensity columns -> Maternal + Fetal Pulsation",
    "comments": "Data pipeline intended for synthetic ppg generation. Includes both maternal and fetal pulsation. I1 is the stationary case, I2 is fetal pulsation, I3 is maternal pulsation and I4 is a mix(currently we have no use for it)",
}

# Save data and config
# ==========================================================================================
data.to_pickle(out_dest)

with open(config_dest, "w+", encoding="utf-8") as outfile:
    json.dump(config, outfile)